### 139. Word Break

#### Dynamic Programming (Trie)

* 時間複雜度：$O(n \cdot L + m)$
* 空間複雜度：$O(2n + m)$ -> $O(n + m)$

1. $n$ 為 字串(s) 總長度。
2. $L$ 為 max_word_length（字典中最長單字的長度）。
3. $m$ 是 word_set 儲存字典單字所佔用的空間。

In [1]:
from typing import List

# 定義 Trie (字典樹) 的節點結構
class TrieNode:
    def __init__(self):
        # 儲存子節點，Key 為字元，Value 為 TrieNode
        self.children = {}
        # 標記是否為單字的結尾
        self.is_end = False

class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:
        # --- 步驟 1：建立 Trie  (time: O(m), space: O(m)) ---
        root = TrieNode() # space: O(m)，儲存字典單字所佔用的空間
        for word in wordDict: # time: O(m)，字典中所有單字字元的總長度
            node = root
            for char in word: # time: O(L)，字典中最長單字的長度，但已經包含在 m 裡了
                if char not in node.children:
                    node.children[char] = TrieNode()
                node = node.children[char]
            node.is_end = True

        n = len(s)
        memory = {} # 建立記憶化快取 # space: O(n)，字串總長度

        # --- 步驟 2：DFS 搜尋  (time: O(n x L), space: O(n)) ---
        # time: O(n)，最大會遍歷字串 s 的每個字，
        # space: O(n)，遞迴最大深度會是字串 s 的長度
        def dfs(start):
            # 終止條件：起始索引到達末端，代表成功拆分
            if start == n:
                return True
            
            # 檢查記憶化快取：避免重複計算相同的 start (總共 n 個狀態)
            if start in memory:
                return memory[start]
            
            # 搜尋起點：重置指標到 Trie 根節點
            node = root 

            # 從 start 開始嘗試向後比對 # time: O(L)，最多比對典中最長單字長度 L 個字元，因為不在字典會自動跳出迴圈
            for i in range(start, n):
                char = s[i]

                # 若 Trie 中無此字元，代表這條路徑無法組成字典單字，直接中斷 (剪枝)
                if char not in node.children:
                    break
                
                # 指標向下移動 (time: O(1)，取代了切片與雜湊計算)
                node = node.children[char]

                # 若匹配到字典單字，且後續路徑也能成功拆分
                if node.is_end and dfs(i + 1):
                    memory[start] = True
                    return True

            # 嘗試過所有可能的 end 點皆失敗
            memory[start] = False
            return False

        # 從索引 0 開始執行深度優先搜尋
        return dfs(0)

In [2]:
s = "applepenapple"; wordDict = ["apple","pen"]
Solution().wordBreak(s, wordDict)

True

#### Dynamic Programming (Top-Down)

* 時間複雜度：$O(n \cdot L^2 + m)$
* 空間複雜度：$O(2n + m)$ -> $O(n + m)$

1. $n$ 為 字串(s) 總長度。
2. $L$ 為 max_word_length（字典中最長單字的長度）。
3. $m$ 是 word_set 儲存字典單字所佔用的空間。

In [3]:
from typing import List

class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:
        # 將 wordDict 轉換為集合 (Set)，加速
        word_set = set(wordDict) # space: O(m)
        
        # 取得字典中最長單字的長度，用來限制後續 DFS 的搜尋範圍（剪枝優化）
        max_word_length = max([len(word) for word in word_set]) if len(word_set) else 0  # time: O(m)
        
        # 建立記憶化字典，紀錄每個 start 索引點的搜尋結果，避免重複計算
        memory = {} # 建立記憶化快取 # space: O(n)，字串總長度        

        # time: O(n)，會遍歷字串 s 的每個字
        # space: O(n)，遞迴最大深度會是字串 s 的長度
        def dfs(start):
            # 終止條件：如果起始索引已到達字串末端，表示前面的拆分全部成功
            if start == len(s):
                return True

            # 檢查記憶化快取：如果這個位置已經算過，直接回傳 True 或 False
            if start in memory:
                return memory[start]

            # 優化：計算 end 的上限，不需檢查超過字典最長單字的長度
            tmp_length = min(start + max_word_length + 1, len(s) + 1)
            
            # 從當前 start 位置開始嘗試各種可能的結束點 end # time: O(L)，範圍不超過字典中最長單字的長度
            for end in range(start + 1, tmp_length):
                # 擷取目前的子字串
                temp_word = s[start:end] # time: O(L) -> 切片長度最長可能達到O(L)

                # 如果子字串存在於字典中，且剩餘的字串 (從 end 開始) 也能成功拆分
                if (temp_word in word_set) and dfs(end):
                    # 紀錄目前 start 點為可行，並向上回傳成功
                    memory[start] = True
                    return True

            # 如果嘗試了所有可能的 end 都無法成功，則紀錄此 start 為失敗
            memory[start] = False
            return False

        # 從字串索引 0 的位置開始進行深度優先搜尋
        return dfs(0)

In [4]:
s = "applepenapple"; wordDict = ["apple","pen"]
Solution().wordBreak(s, wordDict)

True

#### Dynamic Programming (Bottom-Up)

* 時間複雜度：$O(n \cdot L^2 + m)$
* 空間複雜度：$O(m + n)$
  
1. n 為 字串(s) 總長度
2. . $L$ 為 max_word_length（字典中最長單字的長度）。
3. $m$ 是 word_set 儲存字典單字所佔用的空間。

In [5]:
from typing import List

class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:
        # 將 wordDict 轉換為集合 (Set)，加速
        word_set = set(wordDict) # space: o(m)，m 是字典中所有單字的總長度

        # 取得字典中最長單字的長度
        max_word_length = max([len(word) for word in word_set]) if len(word_set) else 0  # time: O(m)

        n = len(s)
        
        # dp[i] 表示 s 的前 i 個字元是否可以被拆分成字典中的單字
        dp = [False] * (n + 1) # space: O(n)
        
        # 初始狀態：空字串可以被拆分
        dp[0] = True
        
        # 遍歷字串的每一個結尾位置 i
        for i in range(1, n + 1): # time: o(n)
            # 檢查從 j 到 i 的子字串
            # 不需要從 0 開始，只需要檢查 i 之前的 max_len 範圍即可，超過 max_len 的子字串絕對不可能在字典裡
            tmp_start = max(0, i - max_word_length)
            for j in range(tmp_start, i): # time: o(L)
                # 如果 s[0:j] 可以拆分，且 s[j:i] 在字典中
                # time: O(L) -> 切片長度最長可能達到O(L)
                if dp[j] and s[j:i] in word_set:
                    dp[i] = True
                    # 只要找到一種可能的拆分方式，就可以停止檢查目前的 i
                    break
                    
        # 回傳整個字串是否可以拆分
        return dp[n]

In [6]:
s = "applepenapple"; wordDict = ["apple","pen"]
Solution().wordBreak(s, wordDict)

True